<a href="https://colab.research.google.com/github/wjdgoruds2/Machine_Learning/blob/main/Assignment14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers#transformers라이브러리 설치
import numpy as np#배열 위함
import pandas as pd#파일 불러옴
from sklearn.model_selection import train_test_split#테스트 학습데이터 분리
from sklearn.linear_model import LogisticRegression#로지스틱 회귀
from sklearn.model_selection import GridSearchCV,cross_val_score#모형 최적화.교차 검증
import torch#torch사용
import transformers as ppb#transformers라이브러리
import warnings#경고 제어
warnings.filterwarnings('ignore')#경고메세지 숨기기

df=pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)#해당 링크에서 데이터셋 읽고 데이터 프레임에 로드
batch_1=df[:2000]#2000문장 사용
print(batch_1.head(3))#batch_1문장중 앞의 세문장 출력

print(batch_1[1].value_counts())#positive,negative문장 개수 출력
model_class=ppb.BertModel#bert모델 사용 정의
tokenizer_class=ppb.BertTokenizer#bert사용해 토큰화 정의
pretrained_weights='bert-base-uncased'#가중치 정의

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)#미리 학습된 데이터 토큰 정의
model = model_class.from_pretrained(pretrained_weights)#미리 학습된 데이터 모델 정의

tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))#문장을 토큰화
print(tokenized)#토큰화 한 결과 출력

max_len = 0#최대 길이 초기화
for i in tokenized.values:#토큰화된 값만큼 반복
    if len(i) > max_len:#각 값의 끝까지 반복
        max_len = len(i)#최대 길이 설정

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])#리스트가 아닌 2-d배열로 패딩 정의
print(padded)#패딩 정보 출력

attention_mask = np.where(padded != 0, 1, 0)#입력을 처리할 때 추가한 패딩 무시 위한 마스크
print(attention_mask.shape)#attention_mask정보 출력

input_ids = torch.tensor(padded) #padded배열 tensor변환
attention_mask = torch.tensor(attention_mask)#attention_mask배열 tensor변환

with torch.no_grad():#가중치 초기화
    last_hidden_states = model(input_ids, attention_mask=attention_mask)#bert를 통해 처리한 결과 정의

features = last_hidden_states[0][:,0,:].numpy()#문장에 대한 특징 벡터
labels = batch_1[1]#라벨 정의
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)#학습과 테스트 데이터로 분리

lr_clf = LogisticRegression()#로지스틱 회귀 모델 정의
lr_clf.fit(train_features, train_labels)#학습 진행

print('accuracy:',lr_clf.score(test_features, test_labels))#테스트 데이터를 통한 학습 결과 분류 정확도 출력



                                                   0  1
0  a stirring , funny and finally transporting re...  1
1  apparently reassembled from the cutting room f...  0
2  they presume their audience wo n't sit still f...  0
1    1041
0     959
Name: 1, dtype: int64


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0       [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1       [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2       [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3       [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4       [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                              ...                        
1995    [101, 2205, 20857, 1998, 11865, 16643, 2135, 5...
1996    [101, 2009, 2515, 1050, 1005, 1056, 2147, 2004...
1997    [101, 2023, 2028, 8704, 2005, 1996, 11848, 199...
1998    [101, 1999, 1996, 2171, 1997, 2019, 9382, 1898...
1999    [101, 1996, 3185, 2003, 25757, 2011, 1037, 244...
Name: 0, Length: 2000, dtype: object
[[  101  1037 18385 ...     0     0     0]
 [  101  4593  2128 ...     0     0     0]
 [  101  2027  3653 ...     0     0     0]
 ...
 [  101  2023  2028 ...     0     0     0]
 [  101  1999  1996 ...     0     0     0]
 [  101  1996  3185 ...     0     0     0]]
(2000, 59)
accuracy: 0.866
